In [ ]:
#v.20251105
#brian maier
#for developing fire business thresholds using fire history or heat detection data and fems nfdrs
import pickle
import geopandas as gpd
#for analysis data plot
from matplotlib import pyplot as plt
import pandas as pd

project_crs = 4326

In [ ]:
#-----user input-----#

#project directory
cwd = 'C:\\1_Projects\\NfdrsRocFems\\r6\\'
#required input directories
#fdra shapefile dir unless using agol rest then None
dir_fdra = None #cwd + 'GB_FDRAs\\GB_FDRAs.shp'
#fires fod dir (karen short gdb), uses version through 2020, fod fires function may need changed if vars change in next fod version
dir_fod_gdb = 'C:\\1_Projects\\SpatialData\\FPA_FOD_20221014.gdb'
#fires inform dir (default csv output from inform filtered starting after fod fires, i.e. 2021-2024), set to None if only using fod gdb
#!note currently no check to make sure fod and inform do not overlap, make sure inform data does not overlap with fod data
dir_inform_csv = cwd + 'Irwin_2021_2024.csv'

#heat detection data if desired for roc analysis, 'Modis', 'Viirs' or None, note modis starts 2005 and viirs 2012
modis_or_viirs = 'Modis'
#set to False if using 1300, True if using daily max
use_min_max = False
#for datetime conversions from utc
tz_local = 'US/Pacific'
#for analysis and climo, start and end year
por_st_dt = 2005
por_end_dt = 2024
#fuel models for analysis
fuel_models_interest = ['Y', 'Z']#'V', 'W', 'X', 
#buffer fdra for raws selection in miles, 0 for no buffer
fdra_buffer_raws = 0

#!since no data standard yet for fdra user will need to do something here if not pnw fdra (washington or oregon)

#!pnw fdra only
#function used to get pnw fdra from nifc
def geojson_rest(url):
    import urllib
    req = urllib.request.urlopen(url)
    gjson = req.read().decode('utf-8')
    gdf = gpd.read_file(gjson)
    return gdf
    
#get pnw fdra from agol rest
def get_fdra():
	url = 'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Pacific_Northwest_Fire_Danger_Rating_Areas_(public)/' \
		'FeatureServer/0/query?where=%27FID%27+%3E+%27-1%27&outFields=*&f=pgeojson&token='
	gdf = geojson_rest(url)
	return gdf
gdf_fdra = get_fdra().to_crs(project_crs)
gdf_fdra['Name'] = gdf_fdra['ParentDocName'] + ' ' + gdf_fdra['FDRAName']

#!other than pnw fdra, get fdra from dir_fdra
#!fdra must have attributes 'ParentDocName' and 'FDRAName' and 'Name' which is unique
#gdf_fdra = gpd.read_file(dir_fdra).to_crs(project_crs)
#print(gdf_fdra.columns)
#do something here to ensure above attributes exist
#gdf_fdra[['ParentDocName', 'FDRAName']] = gdf_fdra.FdraName.str.split('-', expand=True)
#nothing to do here
#gdf_fdra['Name'] = gdf_fdra['ParentDocName'] + ' ' + gdf_fdra['FDRAName']
#-----end user input-----#

#send user vars to roc_main.py
user_vars_1 = {'cwd': cwd, 'dir_fdra': dir_fdra, 'dir_fod_gdb': dir_fod_gdb, 'dir_inform_csv': dir_inform_csv, 'modis_or_viirs': modis_or_viirs, \
    'use_min_max': use_min_max, 'tz_local': tz_local, 'por_st_dt': por_st_dt, 'por_end_dt': por_end_dt, \
     'fuel_models_interest': fuel_models_interest, 'fdra_buffer_raws': fdra_buffer_raws}
with open('user_vars_1.pkl', 'wb') as f:
    pickle.dump({'user_vars_1': user_vars_1}, f)

In [ ]:
from roc_main import AnalysisData
analysis_data_all = AnalysisData(gdf_fdra)
analysis_data_all.add_fires()
analysis_data_all.add_heat_detections()
analysis_data_all.add_raws()
#analysis_data_all also has attributes .fdra, .fires, .raws and .heat_detections (if not None)

In [ ]:
analysis_data_all.fdra.Name.sort_values().unique()

In [ ]:
#!start here again for another fdra, no need to init AnalysisData again
#-----user input-----#
#copy and paste fdra name from list above
fdra_name = 'Northwest Oregon Willamette'
#-----end user input-----#
gdf_fdra_select = gdf_fdra.loc[gdf_fdra.Name == fdra_name]

In [ ]:
from roc_main import FdraAnalysisData
analysis_data = FdraAnalysisData(gdf_fdra_select, analysis_data_all)
analysis_data.add_nfdrs_por()
analysis_data.make_station_map()
analysis_data.make_targets_map()
#analysis_data also has attributes .fdra, .fires, .raws and .heat_detections (if not None) and heat_detections_grouped (if not None)

In [ ]:
#-----user input-----#
#use fire history or heat detections, options are 'Fires' (fires history) or 'Heat' (modis or viirs heat detections)
#! user input modis_or_viirs must not be None to use 'Heat'
fires_or_heat = 'Fires'
#size thresholds for target classes, !recommend 50,90,97 or 50,95,99 for fires, start with 50,90,97
#try starting with 50,80,95 for modis, heat detections seem more sensitive to analysis region than fires
fire_size_percentiles = [50, 90, 97]
heat_size_percentiles = [50, 80, 95]
#enter n splits or use 'auto', !n splits must be smaller then n fires in largest class, recommend starting with 'auto'
#outlier removal, if used, will remove fires and user will need to manually adjust kfold_nsplits
#if manual entry recommend (contamination is not None) rounded n like 5, 10, 20, 30 
#(i.e.if 36 fires in largest class use 30, or try something like 20 if outliers removed)
kfold_nsplits = 'auto'
#outlier removal recommend None, then 'auto', then maybe 0.1 or 0.2 but probably not
contamination = None #'auto'
#-----end user input-----#

#send user vars to roc_main.py
user_vars_2 = {'fires_or_heat': fires_or_heat, 'fire_size_percentiles': fire_size_percentiles, 'heat_size_percentiles': heat_size_percentiles, \
    'kfold_nsplits': kfold_nsplits, 'contamination': contamination}
with open('user_vars_2.pkl', 'wb') as f:
    pickle.dump({'user_vars_2': user_vars_2}, f)

In [ ]:
from roc_main import RocAnalysis
analysis_results = RocAnalysis(analysis_data)
#analysis_results also has attributes .results (if roc_calc_kfold has been called)

In [ ]:
#options are 'ec', 'ic', 'sc', 'bi', and 'sfdi'
analysis_results.roc_calc_kfold('ec')
analysis_results.roc_calc_kfold('bi')
analysis_results.roc_calc_kfold('sfdi')

In [ ]:
#this method finds matches from results with non overlapping thresholds and at least 5 percentile decision space between
#thresholds and then increases decision space until <= n results remain
indice_matches = ['ec', 'bi']
analysis_results.refine_results_decision_space(indice_matches)